In [31]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

In [54]:
def check(first_table,second_table):
    company_A= pd.DataFrame()
    company_A["original_first_table"]=first_table["Participant Name"]
    company_A["suffix"]=first_table["Participant Name"].dropna().apply(lambda x: [y.lower().capitalize() for y in x.split(' ')])
    company_A["suffix"]=company_A["suffix"].apply(lambda x: list(filter(lambda y:y, map(lambda y:re.sub(r'\((.*)\)', '', y), x))))
    company_A["suffix"]=company_A["suffix"].apply(lambda x: ' '.join(y for y in x))
    company_A["suffix"]=company_A["suffix"].apply(str)
    company_A=company_A.drop_duplicates()
    company_A= company_A.dropna()
    company_A= company_A.reset_index(drop=True)

    company_B= pd.DataFrame()
    company_B["original_second_table"]=second_table["Participant Name"]
    company_B["suffix"]=second_table['Participant Name']#.apply(lambda x: x.split('-')[0])
    company_B["suffix"]=company_B["suffix"].apply(lambda x: [y.lower().capitalize() for y in x.split(' ')])
    company_B["suffix"]=company_B["suffix"].apply(lambda x: list(filter(lambda y:y, map(lambda y:re.sub(r'\((.*)\)', '', y), x))))
    company_B["suffix"]=company_B["suffix"].apply(lambda x: ' '.join(y for y in x))
    company_B["suffix"]=company_B["suffix"].apply(str)
    company_B["suffix"]=company_B["suffix"].drop_duplicates()#cannot reset_index: nans are duplicated, are stacked at the bottom
    #values are not nans are stacked on the top with wrong index
    #company_B=company_B[company_B["suffix"].notnull()].reset_index(drop=True)#drop rows where suffix is null
    company_B = company_B.dropna()
    company_B = company_B.reset_index(drop=True)

    '''1. common words'''
    #1.common words
    list_words=company_A['suffix'].apply(str).values.tolist()+company_B['suffix'].apply(str).values.tolist()
    string_words=' '.join(list_words)
    list_word = string_words.split(' ')#from list_words to list_word: split words in each company name
    df_word = pd.DataFrame({'list_word':list_word})
    #df_word["list_word"].value_counts()
    df_word['frequency'] = df_word.groupby('list_word')['list_word'].transform('count')
    df_word=df_word.sort_values(by=['frequency'],ascending=False)
    df_word=df_word.drop_duplicates(['list_word']).reset_index()
    common_words=df_word['list_word'][:8].astype(str).values.tolist()#top 10 frequent words
    # removelist=["Fund","Trust"]
    # common_words = [word for word in common_words if word not in removelist]
    common_words='\s'+"(?=\s)|\s".join(common_words)+"(?=\s)"#escape"." or it will match all character in regular expression
    #common_words=common_words.replace(".","[.]")
    #common_words

    '''2. define N-grams'''
    #2.Define N-grams
    def ngrams(string, n=3):#n=5
        string = re.sub(r'[/]the|[,-./]',r' ', string)#\sOn(?=\s)|\sAviation(?=\s) #r" "
        string=re.sub(r'\s{1,}(?=[.,])|\s{1,}(?=\s)','',string)
        ngrams = zip(*[string[i:] for i in range(n)])
        return [''.join(ngram) for ngram in ngrams]

    '''3. Tf_idf Matrix'''
    #3.Tf_idf Matrix
    company_names1=company_A['suffix']
    company_names2=company_B['suffix']
    company_names=pd.concat([company_names1,company_names2],ignore_index=True)
    company_names='  '+company_names.str[:]+"  "
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix = vectorizer.fit_transform(company_names)
    tf_idf_matrix1=tf_idf_matrix[0:len(company_names1),:]
    tf_idf_matrix2=tf_idf_matrix[len(company_names1):,:]

    '''4. Get Similarity and Matches'''
    #4.Get Similarity and Matches
    def awesome_cossim_top(A, B, ntop, lower_bound=0):
        # force A and B as a CSR matrix.
        # If they have already been CSR, there is no overhead
        A = A.tocsr()
        B = B.tocsr()
        M, _ = A.shape
        _, N = B.shape

        idx_dtype = np.int32

        nnz_max = M*ntop

        indptr = np.zeros(M+1, dtype=idx_dtype)
        indices = np.zeros(nnz_max, dtype=idx_dtype)
        data = np.zeros(nnz_max, dtype=A.dtype)

        ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)

        return csr_matrix((data,indices,indptr),shape=(M,N))

    def get_matches_df(sparse_matrix,name_vector1,name_vector2,top=10000):
        non_zeros = sparse_matrix.nonzero()

        sparserows = non_zeros[0]
        sparsecols = non_zeros[1]

        if top:
            n_matches = top
        else:
            n_matches = sparsecols.size

        left_side = np.empty([n_matches], dtype=object)
        right_side = np.empty([n_matches], dtype=object)
        similairity = np.zeros(n_matches)

        for index in range(0, n_matches):
            left_side[index] = name_vector1[sparserows[index]]
            right_side[index] = name_vector2[sparsecols[index]]
            similairity[index] = sparse_matrix.data[index]


        return pd.DataFrame({'left_side': left_side,'right_side': right_side,'similairity': similairity})

    matches = awesome_cossim_top(tf_idf_matrix1, tf_idf_matrix2.transpose(),100, 0.5)
    matches_df = get_matches_df(matches,company_names1,company_names2,top=matches.count_nonzero())#top = 1000 if required 
    #be careful about the index of the two company list

    #First Half Result and Second Half Result
    result=matches_df.sort_values(['similairity'],ascending=False)
    result=result.drop_duplicates(['left_side'])
    result1=result[result.similairity>0.75]#first half result
    result2=result[result.similairity<=0.75]#second half result
    #result2.reset_index(drop=True).head()
    #result.to_csv('result.csv')

    if len(result2):
        common_words2=df_word['list_word'][:150].astype(str).values.tolist()#top 120 frequent words 
        #in case high similarity caused by common words
        common_words2='\s'+"(?=\s)|\s".join(common_words2)+"(?=\s)"
        common_words2=common_words2.replace(".","[.]")#escape"." or it will match all character in regular expression
        def ngrams(string, n=3):
            string = re.sub(r'[,-./]|'+common_words2,'', string)
            ngrams = zip(*[string[i:] for i in range(n)])
            return [''.join(ngram) for ngram in ngrams]

        result_names= pd.concat([result2.left_side, result2.right_side], ignore_index=True)
        result_names1=result2['left_side'].reset_index(drop=True)#if don't reset_index, the get_matches_df can't work
        result_names2=result2['right_side'].reset_index(drop=True)
        result_names=' '+result_names.str[:]+" "#from two spaced to one
        vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)#analyzer: get_stop_words and build_tokenizer
        tf_idf_matrix = vectorizer.fit_transform(result_names)

        null_company=[]
        for i in range(tf_idf_matrix.shape[0]):
            if len(tf_idf_matrix[i,:].toarray().nonzero()[1])<=12:
                # count_nonzero of four spaces and only five characters is 7 according to result check
                # too large will omit some different "XX technologies" to match
                null_company.append(result_names[i])
        #null_company

        result_clean= result_names.apply(lambda x: "".join(re.sub(r'[’]|'+common_words2,'',x))if x not in null_company else x)
        #result_clean #save all null_company and clean other companies

        vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
        tf_idf_matrix = vectorizer.fit_transform(result_clean)
        tf_idf_matrix1=tf_idf_matrix[0:len(result_names1),:]
        tf_idf_matrix2=tf_idf_matrix[len(result_names1):,:]

        matches2=awesome_cossim_top(tf_idf_matrix1,tf_idf_matrix2.transpose(), 100, 0.75)#0.70
        matches_df2 = get_matches_df(matches2,result_names1,result_names2,top=matches2.count_nonzero())

        result2=matches_df2.sort_values(['similairity'],ascending=False).drop_duplicates()#[matches_df2['similairity']<0.999999]
        #result2

        result_all= pd.concat([result1, result2], ignore_index=True)
        # result_all['right_side'].unique().count()
        # missing= list(set(tobacco_whole)-set(tobacco))
    else:
        result_all = result1.copy()

    result_all=result_all.rename(columns = {'left_side':'suffix'})
    result_all=result_all.merge(company_A, on='suffix', how='left')
    result_all=result_all.rename(columns = {'suffix':'suffix_first_table'})

    result_all=result_all.rename(columns = {'right_side':'suffix'})
    result_all=result_all.merge(company_B, on='suffix', how='left')
    result_all=result_all.rename(columns = {'suffix':'suffix_second_table'})

    result_all=result_all.drop_duplicates(subset=['suffix_second_table']).reset_index(drop=True)

    first_table=first_table.rename(columns={"Participant Name":"original_first_table"})
    second_table=second_table.rename(columns={"Participant Name":"original_second_table"})

    result_all=result_all.merge(first_table, on='original_first_table', how='outer')
    result_all=result_all.merge(second_table, on='original_second_table', how='outer')
    
    return result_all

In [2]:
!ls

Company Master Table Project Documentation.docx
Country_Table_Master_Builder.ipynb
IB_quandl_union.csv
UNCTAD RATIO RANK.csv
UNCTAD SCORE RANK.csv
UNGC_Sentiment_Rank.csv
UNGC_Transparency_Rank.csv
Untitled.ipynb
Untitled1.ipynb
_result_.csv
company match_quandl_IB.ipynb
four_tables.csv
panel.csv
using_total_list.ipynb


In [22]:
total = pd.read_excel('IB_quandl_union.xlsx',index_col=0)

In [23]:
s_name=[]
ind = list(zip(total['original_quandl'].notna(),total['original_IB'].notna()))
for i in range(len(ind)):
    if ind[i][0]:
        s_name.append(total['original_quandl'][i])
    elif ind[i][1]:
        s_name.append(total['original_IB'][i])

In [24]:
total['Participant Name']=s_name

In [25]:
total['Participant Name'].isna().sum()

0

In [28]:
total = total.drop(columns=['suffix_quandl','suffix_IB','similairity','original_quandl','original_IB'])

In [20]:
data_panel = pd.read_csv('panel.csv',index_col=0)
data_UN = pd.read_csv('four_tables.csv',index_col=0)

d:\program files\python37\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,2,30,31,32,33,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
d:\program files\python37\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [40]:
res1 = data_panel.merge(total,left_on='Ticker',right_on='ticker',how='right')

In [44]:
res1.shape,total.shape,data_panel.shape

((3079729, 95), (40805, 43), (1939516, 52))

In [50]:
res1['Participant Name_x'].isna().sum(), res1['Participant Name_y'].isna().sum()

(64011, 0)

In [52]:
res1 = res1.drop(columns=['Participant Name_x'])
res1 = res1.rename(columns={'Participant Name_y':'Participant Name'})

In [55]:
res2 = check(res1,data_UN)

In [57]:
res2.to_csv('IB_quandl_UN.csv')

In [59]:
res2.shape, res2.shape[0]-res1.shape[0], data_UN.shape

((3082770, 134), 3041, (3766, 37))

In [60]:
res2

,suffix_first_table,suffix_second_table,similairity,original_first_table,original_second_table,date,tic,gvkey,PERMNO,equity,...,Rating__UNGC_Transparency_Rank,Organization Id__UNGC_Transparency_Rank,Transparency Rank__UNGC_Transparency_Rank,Transparency Percentage__UNGC_Transparency_Rank,Percentile__UNGC_Transparency_Rank,rank__UNGC_Sentiment_Rank,Avg Sentiment Rank__UNGC_Sentiment_Rank,Organization Id__UNGC_Sentiment_Rank,Avg. Sentiment__UNGC_Sentiment_Rank,PERCENTILE__UNGC_Sentiment_Rank
0,Lietuvos Energijos Gamyba Ab,"Lietuvos Energijos Gamyba, Ab",1.0,LIETUVOS ENERGIJOS GAMYBA AB,"Lietuvos energijos gamyba, AB",NaN,NaN,NaN,NaN,NaN,...,1361.0,17064.0,AA,0.177066,0.638778,2535.0,B,17064.0,0.071214,0.326959
1,Lundin Petroleum Ab,Lundin Petroleum Ab,1.0,LUNDIN PETROLEUM AB,Lundin Petroleum AB,NaN,NaN,NaN,NaN,NaN,...,501.0,11629.0,AAA,0.280776,0.867198,2035.0,B,11629.0,0.082550,0.459761
2,Pandora A/s,Pandora A/s,1.0,Pandora A/S,Pandora A/S,NaN,NaN,NaN,NaN,NaN,...,1045.0,11593.0,AA,0.209949,0.722709,1362.0,A,11593.0,0.096956,0.638513
3,Pandora A/s,Pandora A/s,1.0,Pandora A/S,Pandora A/S,NaN,NaN,NaN,NaN,NaN,...,1045.0,11593.0,AA,0.209949,0.722709,1362.0,A,11593.0,0.096956,0.638513
4,Gabriel Resources Ltd,Gabriel Resources Ltd.,1.0,GABRIEL RESOURCES LTD,Gabriel Resources Ltd.,NaN,NaN,NaN,NaN,NaN,...,2173.0,4088.0,A,0.115514,0.423108,1415.0,A,4088.0,0.095927,0.624436
5,Gabriel Resources Ltd,Gabriel Resources Ltd.,1.0,GABRIEL RESOURCES LTD,Gabriel Resources Ltd.,NaN,NaN,NaN,NaN,NaN,...,2173.0,4088.0,A,0.115514,0.423108,1415.0,A,4088.0,0.095927,0.624436
6,Bolsas Y Mercados Espanoles,Bolsas Y Mercados Espanoles,1.0,BOLSAS Y MERCADOS ESPANOLES,Bolsas y Mercados Espanoles,NaN,NaN,NaN,NaN,NaN,...,746.0,13406.0,AAA,0.246206,0.802125,1358.0,A,13406.0,0.097045,0.639575
7,Bolsas Y Mercados Espanoles,Bolsas Y Mercados Espanoles,1.0,BOLSAS Y MERCADOS ESPANOLES,Bolsas y Mercados Espanoles,NaN,NaN,NaN,NaN,NaN,...,746.0,13406.0,AAA,0.246206,0.802125,1358.0,A,13406.0,0.097045,0.639575
8,Axfood Ab,Axfood Ab,1.0,AXFOOD AB,Axfood AB,NaN,NaN,NaN,NaN,NaN,...,408.0,1051.0,AAA,0.296796,0.891899,2272.0,B,1051.0,0.076813,0.396813
9,Hyflux Ltd,Hyflux Ltd,1.0,HYFLUX LTD,Hyflux Ltd,NaN,NaN,NaN,NaN,NaN,...,1885.0,11295.0,A,0.134907,0.499602,2024.0,B,11295.0,0.082728,0.462683
